In [1]:
import os
from pymongo import MongoClient


In [2]:
import datetime 
# timestamp from unix to 
ts = datetime.datetime.utcfromtimestamp(int(float("1641854884.0")))

In [3]:
ts

datetime.datetime(2022, 1, 10, 22, 48, 4)

In [6]:
db["events"].insert_one({
    "timestamp": ts
})

In [5]:
client = MongoClient(
    os.environ.get("MONGO_DRIVER") + "://" + 
    os.environ.get("MONGO_USERNAME") + ":" + 
    os.environ.get("MONGO_PASSWORD") + "@" +
    os.environ.get("MONGO_HOST") + 
    #":" + os.environ.get("MONGO_PORT") +
    "/" + os.environ.get("MONGO_DB_NAME") +
    "?" + os.environ.get("MONGO_CX_OPTIONS")
)
db = client["dragonfly"]

In [6]:
# consumer for results
import json 
import bson
import kafka_consumer

consumer = kafka_consumer.initialize(
    topic_env='TOPIC_RESULTS',
    group_id_env='CONSUMER_GROUP_CHUNKS'
)
for event in consumer:
    print(event)

ConsumerRecord(topic='results', partition=0, offset=0, timestamp=1641689359767, timestamp_type=0, key='61da310fccb0a1b5eeca72aa', value='{"chunk_id": "61da310fccb0a1b5eeca72aa", "project_id": "000000000000000000000000", "text": "AB eats their lunch on the automation front. I\\u2019ve done both. Siemens is getting better and it\\u2019s starting to not matter as much as it did, but AB really has the tech support and integration piece figured out and have for a long time. The Encompass partnership program is helping AB stay ahead on the 3rd party integration as well. \\n\\nAs far as turbines go, GE makes a much better turbine than Siemens. The GE Mark VI DCS that comes with a F or H class turbine is inferior to Emerson Ovation.", "text_metadata": "{\\"timestamp\\": 1641683943.0, \\"comment_id\\": \\"hru9bqh\\", \\"post_id\\": \\"rzcfdz\\"}", "ner_results": "{\\"GE\\": {\\"span\\": [355, 357]}}", "sa_results": "{\\"label\\": \\"positive\\", \\"score\\": 0.9985002279281616}"}', headers=[], 

KeyboardInterrupt: 

In [11]:
import json
event = json.loads('{"chunk_id": "61da310fccb0a1b5eeca72aa", "project_id": "000000000000000000000000", "text": "AB eats their lunch on the automation front. I\\u2019ve done both. Siemens is getting better and it\\u2019s starting to not matter as much as it did, but AB really has the tech support and integration piece figured out and have for a long time. The Encompass partnership program is helping AB stay ahead on the 3rd party integration as well. \\n\\nAs far as turbines go, GE makes a much better turbine than Siemens. The GE Mark VI DCS that comes with a F or H class turbine is inferior to Emerson Ovation.", "text_metadata": "{\\"timestamp\\": 1641683943.0, \\"comment_id\\": \\"hru9bqh\\", \\"post_id\\": \\"rzcfdz\\"}", "ner_results": "{\\"GE\\": {\\"span\\": [355, 357]}}", "sa_results": "{\\"label\\": \\"positive\\", \\"score\\": 0.9985002279281616}"}')

In [16]:
event

{'chunk_id': '61da310fccb0a1b5eeca72aa',
 'project_id': '000000000000000000000000',
 'text': 'AB eats their lunch on the automation front. I’ve done both. Siemens is getting better and it’s starting to not matter as much as it did, but AB really has the tech support and integration piece figured out and have for a long time. The Encompass partnership program is helping AB stay ahead on the 3rd party integration as well. \n\nAs far as turbines go, GE makes a much better turbine than Siemens. The GE Mark VI DCS that comes with a F or H class turbine is inferior to Emerson Ovation.',
 'text_metadata': '{"timestamp": 1641683943.0, "comment_id": "hru9bqh", "post_id": "rzcfdz"}',
 'ner_results': '{"GE": {"span": [355, 357]}}',
 'sa_results': '{"label": "positive", "score": 0.9985002279281616}'}

In [17]:
for entity in event:
    print(entity)

chunk_id
project_id
text
text_metadata
ner_results
sa_results


In [ ]:
for entity in event["ner_results"]:
    print({
        "entity" : entity,
        "type" : "opinion",
        "origin" : "reddit",
        "value" : event["sa_results"]["label"]
        "timestamp" : event["text_metadata"]["timestamp"],
    })

In [7]:
import requests
import json

In [8]:
request_body = json.dumps({
    "url_list" : [
        {
            "project_id":"000000000000000000000000",
            "url": "https://www.reddit.com/r/stocks/",
            "url_metadata": {"type" : "subreddit"}
        },
        {
            "project_id":"000000000000000000000000",
            "url": "https://www.reddit.com/r/StockMarket/",
            "url_metadata": {"type" : "subreddit"}
        }
    ]
})

In [9]:
r = requests.post("http://api:6969/process_urls",data=request_body)